# YOLOv1
- [paper](https://arxiv.org/pdf/1506.02640.pdf)

### Preview

<img src = "https://github.com/fned49/blog/blob/main/YOLO/figure/figure2.png?raw=true">

- input image를 $S\times S$ grid로 나눕니다.
- object의 center가 특정 grid cell에 속한다면, 해당 grid cell은 object를 탐지하는 역할을 합니다.
- confidence = $Pr\left(Object\right) * IOU_{pred}^{truth}$
- object의 center가 특정 grid cell에 속하지 않으면, 해당 grid cell의 confidence score는 0입니다.
- 각 grid cell은 $C$ conditional class probability $Pr\left(Class_i\vert Object\right)$를 예측합니다.
- box의 갯수에 상관없이 grid cell 당 class probability의 one set을 예측합니다.
- test time에 우리는 conditional class probability와 각 box confidence를 아래와 같이 곱합니다.
- $Pr\left(Class_i\vert Object\right) * Pr\left(Object\right) * IOU_{pred}^{truth} = Pr\left(Class_i\right) * IOU_{pred}^{truth}$
  


YOLO v1은 localization과 classification을 하나의 문제로 정의하여 network가 동시에 두 task를 수행하도록 설계하였습니다.\
이를 위해 이미지를 지정한 grid로 나누고, 각 grid cell이 한번에 bounding box와 class 정보라는 2가지 정답을 도출하도록 만들었습니다.\
또한 각 grid cell에서 얻은 정보를 feature map이 잘 encode할 수 있도록, 독자적인 Convolutional Network인 DarkNet을 도입합니다.\
이를 통해 얻은 feature map을 활용하여, 자체적으로 정의한 regression loss를 통해 전체 모델을 학습시킵니다. 

### Main Ideas

<img src = "https://github.com/fned49/blog/blob/main/YOLO/figure/figure6.jpeg?raw=true">

YOLO v1은 별도의 region proposals를 사용하지 않고 전체 이미지를 입력하여 사용합니다.\
먼저 전체 이미지를 SxS 크기의 grid로 나눠줍니다.\
여기서 객체의 중심이 특정 grid cell에 위치한다면, 해당 grid cell은 그 객체를 detect하도록 할당(responsible for)됩니다.\
위의 그림에서 4행 3열의 grid cell이 왼쪽의 개를 예측하도록 할당되었고, 4행 4열의 grid cell이 오른쪽의 개를 예측하도록 할당되었습니다.\
그리고 나머지 grid cell은 객체를 예측하는데 참여할 수 없음을 의미합니다. 

각각의 grid cell은 B개의 bounding box와 해당 bounding box에 대한 confidence score를 예측합니다.\
confidence score는 해당 bounding box에 객체가 포함되어 있는지 여부와, box가 얼마나 정확하게 ground truth box를 예측했는지를 반영하는 수치입니다.\
만약 grid cell 내에 객체가 존재하지 않는다면 confidence score는 0이 될 것입니다. 반대로 grid cell 내에 객체가 존재한다면 confidence score는 IoU 값과 같아집니다.

<img src = "https://github.com/fned49/blog/blob/main/YOLO/figure/figure7.jpeg?raw=true">

각각의 bounding box는 box의 좌표 정보(x, y, w, h)와 confidence score라는 5개의 예측값을 가집니다.\
여기서 (x, y)는 grid cell의 경계에 비례한 box의 중심 좌표를 의미합니다.\
높이와 너비는 역시 마찬가지로 grid cell에 비례한 값입니다.\
여기서 주의할 점은 x, y는 grid cell 내에 위치하기에 0~1 사이의 값을 가지지만,\
객체의 크기가 grid cell의 크기보다 더 클 수 있기 때문에 width, height 값은 1 이상의 값을 가질 수 있습니다.

하나의 bounding box는 하나의 객체만을 예측하며, 하나의 grid cell은 하나의 bounding box를 학습에 사용합니다.\
각 grid cell별로 B개의 bounding box를 예측한다고 할 때, confidence score가 가장 높은 1개의 bounding box만 학습에 사용하는 것입니다. 

<img src = "https://github.com/fned49/blog/blob/main/YOLO/figure/figure8.png?raw=true">

각 grid cell은 C개의 conditional class probabilities인 $Pr\left(Class_i\vert Object\right)$를 예측합니다.\
이는 특정 grid cell에 객체가 존재한다고 가정했을 때, 특정 $Class_i$일 확률인 조건부 확률값입니다.\
bounding box 수와 상관없이 하나의 grid cell마다 하나의 조건부 확률을 예측합니다.\
여기서 짚고 넘어갈 점은 bounding box별로 class probabilities를 예측하는 것이 아니라, grid cell별로 예측한다는 것입니다. 

논문에서는 $S$=7, $B$=2, $C$=20 설정하였습니다. (PASCAL VOC 데이터셋을 사용하여 학습하였기에 class의 수가 20개)\
즉, 이미지를 7x7 grid로 나누고 각 grid cell은 2개의 bounding box와 해당 box의 confidence score, 그리고 C개의 class probabilities를 예측합니다.\
즉 이미지별 예측값의 크기는 7x7x(2x5+20)입니다.\
이와 같은 과정을 통해 bounding box의 위치와 크기, 그리고 class에 대한 정보를 동시에 예측하는 것이 가능해집니다.

### Architecture

<img src = "https://github.com/fned49/blog/blob/main/YOLO/figure/figure3.png?raw=true">

- network의 convolutional layer는 image의 feature를 추출하며, fc layer는 probability와 coordinate를 예측한다
- 24개의 convolutional layer, 2개의 fc layer
- $1\times 1$ conv layer 다음에 $3\times 3$ conv layer
- final output은 $7\times 7\times 30$ tensor

YOLO v1 모델은 앞서 살펴본 최종 예측값의 크기인 7x7x30에 맞는 feature map을 생성하기 위해 DarkNet이라는 독자적인 Convolutional Network을 설계합니다.\
DarkNet은 ImageNet 데이터셋을 통해 학습시켰습니다. 학습 결과 GoogLeNet 모델과 비슷한 top-5 88% 정도의 정확도를 보입니다.\
이후 모델이 detection task를 수행할 수 있도록 4개의 conv layer와 2개의 fc layer를 추가합니다.\
또한 classification task를 위해 학습시켰을 때는 224x224 크기의 이미지를 사용한 반면,\
detection task를 위한 학습 시에는 이미지의 크기를 키워 448x448 크기의 이미지를 사용한다고 합니다.\
이는 detection task는 결이 고운(fine grained) 시각 정보를 필요로 하기 때문이라고 설명합니다.

### Loss function

- conv layer는 ImageNet-1K dataset으로 pretrain을 진행했다
- 작은 object를 탐지하기 위해 image size를 $224\times 224$부터 $448\times 448$로 점차 늘려갔다  
- output layer는 class probability와 box coordinate를 예측하는데 이때 0~1 normalizaed된 값을 사용한다  
- finaly layer에는 linear activation function을 사용하고, 그 외에는 leaky ReLU activation function을 사용한다  
- SSE(sum-squared error)를 최적화를 한다 -> 이는 최적화하기 편리하지만 AP를 maximizing 하기에는 어려움이 있다  
  
보통 이미지에서 대부분의 grid cell은 object를 포함하지 않는다. 이는 no object의 gradient가 object의 gradient를 압도해버리는 문제가 생긴다.\
이를 개선하기 위해 box coordinate의 weight는 증가시키고 confidence의 weight는 감소시킨다.

- $\lambda_{coord}=5$, $\lambda_{noobj}=0.5$  

SSE에서 large box와 small box에 대한 error의 weight는 동일하다.\
그래서 small box보다 large box의 편차가 덜 중요하다는 것을 loss function에 반영해야 한다.\
이를 부분적으로 해결하기 위해 directly width, height 대신 square root를 씌운 값을 예측한다.

- $\mathcal{1}_{ij}^{obj}$ : $i$번째 cell과 $j$번째 box에 object가 존재 유무
- hyperparameter : batch size = 64, momentum = 0.9, decay = 0.0005 
- learning rate schedule : 75 epoch까지는 $10^{-2}$로 학습하고, 그 이후 30 epoch까지 $10^{-3}$, 마지막 30 epoch은 $10^{-4}$로 학습  
- overfitting 방지로 dropout과 data augmentation을 적용한다
- first fc layer 이후에 dropout rate = 0.5로 적용한다  
- data augmentation으로 random scaling with scale 1.2, random HSV color space를 적용한다  

#### <img src = "https://github.com/fned49/blog/blob/main/YOLO/figure/figure9.png?raw=true">

기존 R-CNN 계열의 모델이 classification, localization task에 맞게 서로 다른 loss function을 사용했던 것과 달리,\
YOLO v1 모델은 regression 시 주로 사용되는 SSE(Sum of Squared Error)를 사용합니다.\
위의 그림에서 볼 수 있듯이 Localization loss, Confidence loss, Classification loss의 합으로 구성되어 있습니다. 

#### Localization loss

$$\lambda_{coord}\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{obj}\left[\left(x_i-\hat{x}_i\right)^2 + \left(y_i-\hat{y}_i\right)^2\right]$$  
$$+\lambda_{coord}\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{obj}\left[\left(\sqrt{w_i}-\sqrt{\hat{w}_i}\right)^2 + \left(\sqrt{h_i}-\sqrt{\hat{h}_i}\right)^2\right]$$ 

- $\lambda_{coord}$ : 많은 grid cell은 객체를 포함하지 않아 confidence score가 0이 되어 객체를 포함하는 grid cell의 gradient를 압도하여, 모델이 불안정해질 수 있습니다.\
$\lambda_{coord}$는 이러한 문제를 해결하기 위해 객체를 포함하는 cell에 가중치를 두는 파라미터입니다. 논문에서는 $\lambda_{coord} = 5$로 설정합니다. 

- $S^2$ : grid cell의 수(7x7=49)
- $B$ : 각 grid cell의 bounding box의 수(=2)
- $\mathcal{1}_{ij}^{obj}$ : $i$번째 grid cell의 $j$번째 bounding box가 객체를 예측하도록 할당(responsible for)받았을 때 1, 그렇지 않을 경우 0인 index parameter입니다.\
  앞서 설명했듯이 grid cell에서는 B개의 bounding box를 예측하지만 그 중 confidence score가 높은 오직 1개의 bounding box만을 학습에 사용합니다.
- $x_i$, $y_i$, $w_i$, $h_i$ : ground truth box의 $x$, $y$ 좌표와 width, height\
 여기서 크기가 큰 bounding box의 작은 오류가 크기가 작은 bounding box의 오류보다 덜 중요하다는 것을 반영하기 위해 $w_i$, $h_i$ 값에 루트를 씌어주게 됩니다. 
- $\hat{x}_i$, $\hat{y}_i$, $\hat{w}_i$, $\hat{h}_i$ : 예측 bounding box의 $x, y$ 좌표, width, height

#### Confidence loss

$$+\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{obj}\left(C_i-\hat{C}_i\right)^2$$  
$$+\lambda_{noobj}\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{noobj}\left(C_i-\hat{C}_i\right)^2$$  

- $\lambda_{noobj}$ : 앞서 언급한 객체를 포함하지 않는 grid cell에 곱해주는 가중치 파라미터입니다. 논문에서는 $\lambda_{noobj}=0.5$로 설정했습니다.\
$\lambda_{obj}$로 설정한 것에 비해 상당히 작게 설정하여 객체를 포함하지 않은 grid cell의 영향력을 줄였습니다. 
- $\mathcal{1}_{ij}^{noobj}$ : $i$번째 grid cell의 $j$번째 bounding box가 객체를 예측하도록 할당(responsible)받지 않았을 때 1, 그렇지 않을 경우 0
- $C_i$ : 객체가 포함되어 있을 경우 1, 그렇지 않을 경우 0
- $\hat{C}_i$ : 예측한 bounding box의 confidence score


#### Classification loss

$$+\sum_{i=0}^{S^2}\mathcal{1}_{i}^{obj}\sum_{c\in classes}\left(p_i\left(c\right)-\hat{p}_i\left(c\right)\right)$$  

- $p_i\left(c\right)$ : 실제 class probabilities
- $\hat{p}_i\left(c\right)$ : 예측한 class probabilities

### Inference  
- YOLOv1은 PASCAL VOC에서 이미지당 98개의 box와 class probability를 예측한다  
- grid design은 bounding box prediction의 다양성을 보여준다  
- 그러나 몇몇 large object 또는 grid cell 경계 근처 object는 multiple cell에 의해 localize가 잘 될 수 있다  

### Limitations of YOLO  
grid cell 당 2개의 box밖에 예측하지 못해 spatial constraint가 존재한다.\
이는 small object 또는 특정 grid cell 안에 여러 개의 object가 존재하는 경우 모델의 성능이 떨어지게 만든다.
- YOLO는 data로부터 학습을 하기 때문에 새롭거나 특이한 aspect ratio 또는 환경설정에서 object detection 성능을 일반화하기 어렵다  
- 마지막으로 loss function을 최적화하는 동안 small box에 대한 error와 large box에 대한 error를 똑같이 취급한다  
- large box의 small error는 일반화가 잘 되는 반면 small box의 small error는 IOU에 훨씬 더 큰 영향을 미친다  
- 이 원인은 잘못된 localization이다  

# YOLOv2
- [paper](https://arxiv.org/pdf/1612.08242.pdf)